In [2]:
import requests
import time
import datetime
import random
from bs4 import BeautifulSoup
import os
import sys
import re
import csv

today = str(datetime.date.today())

class RakutenScraper:
    """
    楽天のカテゴリページや検索結果ページから再帰的にリンクを収集し、
    商品ページURLから「ショップURL」を抽出するサンプルクラス。
    できるだけ多くのショップURLを抽出します。
    """
    def __init__(self, base_url):
        self.base_url = base_url
        self.master_list = []        # 再帰的に収集したリンクを保存
        self.item_page_list = []     # 楽天の商品ページURLのみを格納
        self.shop_url_list = set()   # ショップURL（重複排除）を格納

    def simple_request(self, url, first=False):
        """
        指定URLのページから <a> タグの href を収集し、self.master_list に追加する。
        first=True のときは、取得したリンクリストを返す（初回のみ）。
        """
        print(f"アクセス中: {url} | 既存リンク数: {len(self.master_list)}")
        try:
            r = requests.get(url, timeout=(30.0, 47.5))
        except requests.exceptions.RequestException:
            return False

        if r.status_code == 200 and 'text/html' in r.headers.get('Content-Type', ''):
            soup = BeautifulSoup(r.content, 'html.parser')
            atag_list = soup.find_all('a')
            if first:
                unique_hrefs = []
                for a_tag in atag_list:
                    href = a_tag.get('href')
                    if href and (href not in self.master_list):
                        self.master_list.append(href)
                    if href and (href not in unique_hrefs):
                        unique_hrefs.append(href)
                return unique_hrefs
            else:
                for a_tag in atag_list:
                    href = a_tag.get('href')
                    if href and (href not in self.master_list):
                        self.master_list.append(href)
                return len(self.master_list)
        else:
            return False

    def get_item_url(self):
        """
        master_list に含まれるURLのうち、楽天の商品ページURLを抽出し、
        self.item_page_list を再計算する。
        例: https://item.rakuten.co.jp/ショップコード/商品ID/
        """
        pattern = r'^https://item\.rakuten\.co\.jp/([^/]+)/(\d+)/?$'
        self.item_page_list = []
        for url in self.master_list:
            if url and re.match(pattern, url):
                self.item_page_list.append(url)
        return self.item_page_list

    def extract_shop_urls(self):
        """
        item_page_list の各URLからショップコードを抜き出し、
        https://www.rakuten.co.jp/ショップコード/ の形でショップURLを生成します。
        リミットは設けず、可能な限り多くのショップURLを抽出します。
        """
        pattern = r'^https://item\.rakuten\.co\.jp/([^/]+)/(\d+)/?$'
        shop_urls = set()
        for item_url in self.item_page_list:
            m = re.match(pattern, item_url)
            if m:
                shop_code = m.group(1)
                shop_url = f"https://www.rakuten.co.jp/{shop_code}/"
                shop_urls.add(shop_url)
        self.shop_url_list = shop_urls
        return list(shop_urls)

def get_company_info_from_info_page(shop_url):
    """
    ショップURLに対して末尾にinfo.htmlを付与し、
    固定の企業情報ページHTMLから、
    <dl>内の<dt>タグを調査して「株式会社」を含むテキストを会社名として抽出し、
    会社名の直後には必ず「〒」が続く前提で、その直前までを会社名とみなす。
    また、TEL:の後に続く電話番号を抽出する。
    ※有限会社は抽出対象外です。
    """
    info_url = shop_url.rstrip('/') + '/info.html'
    print(f"取得する企業情報ページ: {info_url}")
    
    try:
        response = requests.get(info_url, timeout=15)
        response.raise_for_status()
    except Exception as e:
        print("企業情報ページの取得に失敗しました:", e)
        return "Not Found", "Not Found"
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    company_name = "Not Found"
    dl_tag = soup.find("dl")
    if dl_tag:
        dt_tags = dl_tag.find_all("dt")
        for dt in dt_tags:
            dt_text = dt.get_text(strip=True)
            # 有限会社は抽出しないため、パターンは「株式会社」から始まるものだけに変更
            pattern = r'(株式会社)([^〒]+)(?=〒)'
            match = re.search(pattern, dt_text)
            if match:
                company_name = (match.group(1) + match.group(2)).strip()
                break
    
    telephone = "Not Found"
    tel_elem = soup.find(text=re.compile("TEL:"))
    if tel_elem:
        match = re.search(r'TEL:\s*([\d\-]+)', tel_elem)
        if match:
            telephone = match.group(1)
    
    return company_name, telephone

if __name__ == "__main__":
    # 既存の企業リスト（ショップURLのみを記載したCSV）のファイルパス
    existing_companies_file = "./rakuten_scraping.csv"
    # 既存のショップURLを読み込む（存在する場合）
    existing_shops = set()
    if os.path.exists(existing_companies_file):
        print(f"既存の企業リスト {existing_companies_file} を読み込みます。")
        with open(existing_companies_file, 'r', encoding='utf-8-sig') as f:
            reader = csv.DictReader(f)
            for row in reader:
                shop_url = row.get("shop_url")
                if shop_url:
                    existing_shops.add(shop_url)
        print(f"既存のショップURL数: {len(existing_shops)}")
    else:
        print("既存の企業リストが見つかりません。")

    # 1) 対象の楽天カテゴリ／検索結果ページのURLを入力
    print("例: https://www.rakuten.co.jp/category/110729/ または https://search.rakuten.co.jp/search/mall/レディースファッション")
    while True:
        input_url = input("スクレイピング対象URLを入力してください: ").strip()
        if not input_url.startswith("https://"):
            print("URLが不正です。先頭がhttps:// で始まる必要があります。\n")
            continue
        else:
            print("スクレイピングを開始します。巡回範囲が大きくなる可能性があるのでご注意ください。")
            break

    scraper = RakutenScraper(input_url)

    # 2) 初回リクエストでリンクを取得
    first_links = scraper.simple_request(input_url, first=True)
    if not first_links:
        print("初回のリクエストでリンクが取得できませんでした。終了します。")
        sys.exit(0)

    # 3) 初回で取得したリンクをさらに巡回してリンクを収集（BFS的に1階層のみ）
    #    ※ユーザーが途中で中断しても、これまでに集まったリンクで後続処理を行います。
    try:
        start_time = time.time()
        for link in first_links:
            # ここでは30分を上限として巡回
            if time.time() - start_time > 1800:
                print("30分経過したため、巡回を終了します。")
                break
            if link.startswith("http"):
                scraper.simple_request(link, first=False)
                time.sleep(random.uniform(1, 1.7))
                # 毎回、商品ページURLとショップURLを更新してチェック
                scraper.get_item_url()
                current_shop_urls = scraper.extract_shop_urls()
                print(f"現在のショップURL数: {len(current_shop_urls)}")
    except KeyboardInterrupt:
        print("\nユーザーによって中断されました。これまでのデータを元に後続の処理を実行します。")
    finally:
        # 最終的に再計算して全てのショップURLを確認
        item_urls = scraper.get_item_url()
        shop_urls = scraper.extract_shop_urls()
        print(f"\n最終的な商品ページURL数: {len(item_urls)}")
        print(f"取得したショップURLの数: {len(shop_urls)}")

        # 4) 各ショップURLに対して末尾にinfo.htmlを付与し、
        #    企業情報ページから企業名と電話番号を取得（既存のショップはスキップ）
        #    ※ここでinfo.html付きのURLも計算して、結果に追加します。
        results = []
        for shop_url in shop_urls:
            if shop_url in existing_shops:
                print(f"{shop_url} は既に抽出済みのため、スキップします。")
                continue
            # info.htmlを付与したURL
            info_url = shop_url.rstrip('/') + '/info.html'
            company_name, telephone = get_company_info_from_info_page(shop_url)
            results.append({
                "shop_url": shop_url,         # info.html付与前のショップURL
                "info_url": info_url,         # info.htmlを付与したURL
                "company_name": company_name,
                "telephone": telephone
            })
            time.sleep(random.uniform(1, 2))
        
        # 5) CSV保存用ディレクトリの作成
        csv_dir = "./csv_data/"
        if not os.path.exists(csv_dir):
            os.makedirs(csv_dir)
        
        # 6) 新たに抽出した企業情報をCSVに書き出し（ショップURL、info_url、企業名、TEL）
        filename = f"{today}-company_info.csv"
        output_path = os.path.join(csv_dir, filename)
        
        # Excelで開いたときに文字化けを防ぐため、BOM付きUTF-8で保存
        with open(output_path, 'w', newline='', encoding='utf-8-sig') as f:
            fieldnames = ["shop_url", "info_url", "company_name", "telephone"]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for row in results:
                writer.writerow(row)
        
        print(f"\n--- CSV出力完了 ---\nファイル: {output_path}")
        
        # ① 既存の企業リスト（ショップURLのみ）の更新：新たに抽出したショップURLも追加して保存
        all_shops = existing_shops.union({res["shop_url"] for res in results})
        with open(existing_companies_file, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=["shop_url"])
            writer.writeheader()
            for shop in all_shops:
                writer.writerow({"shop_url": shop})
        
        print(f"\n--- 既存企業リストの更新完了 ---\nファイル: {existing_companies_file}")
        print("処理が完了しました。")


既存の企業リスト ./rakuten_scraping.csv を読み込みます。
既存のショップURL数: 0
例: https://www.rakuten.co.jp/category/110729/ または https://search.rakuten.co.jp/search/mall/レディースファッション


スクレイピング対象URLを入力してください:  https://search.rakuten.co.jp/search/mall/-/558873/?min=10000


スクレイピングを開始します。巡回範囲が大きくなる可能性があるのでご注意ください。
アクセス中: https://search.rakuten.co.jp/search/mall/-/558873/?min=10000 | 既存リンク数: 0
アクセス中: https://www.rakuten.co.jp/ | 既存リンク数: 456
現在のショップURL数: 7
アクセス中: https://basket.step.rakuten.co.jp/rms/mall/bs/cartall/ | 既存リンク数: 1003
現在のショップURL数: 7
アクセス中: https://event.rakuten.co.jp/notification/ | 既存リンク数: 1008
現在のショップURL数: 7
アクセス中: https://coupon.rakuten.co.jp/myCoupon/%E6%A5%BD%E5%A4%A9%E5%B8%82%E5%A0%B4 | 既存リンク数: 1021
現在のショップURL数: 7
アクセス中: https://ashiato.rakuten.co.jp/rms/sd/ashiato/vc | 既存リンク数: 1021
現在のショップURL数: 7
アクセス中: https://my.bookmark.rakuten.co.jp/item | 既存リンク数: 1027
現在のショップURL数: 7
アクセス中: https://order.my.rakuten.co.jp/ | 既存リンク数: 1027
現在のショップURL数: 7
アクセス中: https://ichiba.faq.rakuten.net/ | 既存リンク数: 1027
現在のショップURL数: 7
アクセス中: https://ichiba.faq.rakuten.net/form/item-guide | 既存リンク数: 1083
現在のショップURL数: 7
アクセス中: https://www.rakuten.co.jp/ec/?camp=701100000008vIp&scid=wi_ich_common | 既存リンク数: 1085


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


現在のショップURL数: 7
アクセス中: https://event.rakuten.co.jp/campaign/supersale/?l-id=search_browse_pc_header_top | 既存リンク数: 1161
現在のショップURL数: 7
アクセス中: https://www.rakuten.co.jp | 既存リンク数: 1308
現在のショップURL数: 7
アクセス中: https://search.rakuten.co.jp/search/mall/-/551177/?min=10000 | 既存リンク数: 1308
現在のショップURL数: 11
アクセス中: https://www.rakuten.co.jp/category/558873/ | 既存リンク数: 1665
現在のショップURL数: 21
アクセス中: https://search.rakuten.co.jp/search/mall/-/558873/?min=10000&used=0 | 既存リンク数: 2056
現在のショップURL数: 21
アクセス中: https://search.rakuten.co.jp/search/mall/-/558873/?min=10000&used=1 | 既存リンク数: 2342
現在のショップURL数: 22
アクセス中: https://search.rakuten.co.jp/search/mall/-/558873/tg5002023/?min=10000 | 既存リンク数: 2656
現在のショップURL数: 22
アクセス中: https://search.rakuten.co.jp/search/mall/-/558873/tg1000886/?min=10000 | 既存リンク数: 2658
現在のショップURL数: 27
アクセス中: https://search.rakuten.co.jp/search/mall/-/558873/tg1000881/?min=10000 | 既存リンク数: 2941
現在のショップURL数: 29
アクセス中: https://search.rakuten.co.jp/search/mall/-/558873/tg1000873/?min=10000 | 既存リンク

C:\Users\to0ar\AppData\Local\Temp\ipykernel_25268\797702007.py:122: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tel_elem = soup.find(text=re.compile("TEL:"))


取得する企業情報ページ: https://www.rakuten.co.jp/benbe/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/hff/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/ash-store/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/ginlet/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/p-mrt/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/qwerty/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/mens-suit/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/deepinsideinc-store/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/esports/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/auc-americanbass/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/freshbox/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/lesnoirgraph/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/acoustic/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/as-stock/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/icefield/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/freshstore/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/elpresidente/info.html

In [8]:
import requests
import time
import datetime
import random
from bs4 import BeautifulSoup
import os
import sys
import re
import csv

today = str(datetime.date.today())

class RakutenScraper:
    """
    楽天のカテゴリページや検索結果ページから再帰的にリンクを収集し、
    商品ページURLから「ショップURL」を抽出するサンプルクラス。
    できるだけ多くのショップURLを抽出します。
    """
    def __init__(self, base_url):
        self.base_url = base_url
        self.master_list = []        # 再帰的に収集したリンクを保存
        self.item_page_list = []     # 楽天の商品ページURLのみを格納
        self.shop_url_list = set()   # ショップURL（重複排除）を格納

    def simple_request(self, url, first=False):
        """
        指定URLのページから <a> タグの href を収集し、self.master_list に追加する。
        first=True のときは、取得したリンクリストを返す（初回のみ）。
        """
        print(f"アクセス中: {url} | 既存リンク数: {len(self.master_list)}")
        try:
            r = requests.get(url, timeout=(30.0, 47.5))
        except requests.exceptions.RequestException:
            return False

        if r.status_code == 200 and 'text/html' in r.headers.get('Content-Type', ''):
            soup = BeautifulSoup(r.content, 'html.parser')
            atag_list = soup.find_all('a')
            if first:
                unique_hrefs = []
                for a_tag in atag_list:
                    href = a_tag.get('href')
                    if href and (href not in self.master_list):
                        self.master_list.append(href)
                    if href and (href not in unique_hrefs):
                        unique_hrefs.append(href)
                return unique_hrefs
            else:
                for a_tag in atag_list:
                    href = a_tag.get('href')
                    if href and (href not in self.master_list):
                        self.master_list.append(href)
                return len(self.master_list)
        else:
            return False

    def get_item_url(self):
        """
        master_list に含まれるURLのうち、楽天の商品ページURLを抽出し、
        self.item_page_list を再計算する。
        例: https://item.rakuten.co.jp/ショップコード/商品ID/
        """
        pattern = r'^https://item\.rakuten\.co\.jp/([^/]+)/(\d+)/?$'
        self.item_page_list = []
        for url in self.master_list:
            if url and re.match(pattern, url):
                self.item_page_list.append(url)
        return self.item_page_list

    def extract_shop_urls(self, existing_shops=None):
        """
        item_page_list の各URLからショップコードを抜き出し、
        https://www.rakuten.co.jp/ショップコード/ の形でショップURLを生成します。
        既存のショップURL（existing_shops）と重複していない場合のみ追加します。
        """
        pattern = r'^https://item\.rakuten\.co\.jp/([^/]+)/(\d+)/?$'
        shop_urls = set()
        for item_url in self.item_page_list:
            m = re.match(pattern, item_url)
            if m:
                shop_code = m.group(1)
                shop_url = f"https://www.rakuten.co.jp/{shop_code}/"
                # 既存のショップURLと重複していなければ追加
                if existing_shops is None or shop_url not in existing_shops:
                    shop_urls.add(shop_url)
        self.shop_url_list = shop_urls
        return list(shop_urls)

def get_company_info_from_info_page(shop_url):
    """
    ショップURLに対して末尾にinfo.htmlを付与し、
    固定の企業情報ページHTMLから、<dl>内の<dt>タグのテキストを取得します。
    そのテキストに「株式会社」が含まれていれば企業名として採用し、
    ※「有限会社」が含まれているものは抽出対象外とします。
    同時にTEL:の後の電話番号も抽出します。
    """
    info_url = shop_url.rstrip('/') + '/info.html'
    print(f"取得する企業情報ページ: {info_url}")
    
    try:
        response = requests.get(info_url, timeout=15)
        response.raise_for_status()
    except Exception as e:
        print("企業情報ページの取得に失敗しました:", e)
        return "Not Found", "Not Found"
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    company_name = "Not Found"
    phone_number = "Not Found"
    
    dl_tag = soup.find("dl")
    if dl_tag:
        dt_tags = dl_tag.find_all("dt")
        for dt in dt_tags:
            dt_text = dt.get_text(strip=True)
            # dtタグのテキストに「株式会社」が含まれており、
            # なおかつ「有限会社」が含まれていなければ企業名として採用
            if "株式会社" in dt_text and "有限会社" not in dt_text:
                company_name = dt_text
                break

    tel_elem = soup.find(text=re.compile("TEL:"))
    if tel_elem:
        match = re.search(r'TEL:\s*([\d\-]+)', tel_elem)
        if match:
            phone_number = match.group(1)
    
    return company_name, phone_number

if __name__ == "__main__":
    # 既存の企業リスト（ショップURLのみを記載したCSV）のファイルパス
    existing_companies_file = "./rakuten_scraping.csv"
    # 既存のショップURLを読み込む（存在する場合）
    existing_shops = set()
    if os.path.exists(existing_companies_file):
        print(f"既存の企業リスト {existing_companies_file} を読み込みます。")
        with open(existing_companies_file, 'r', encoding='utf-8-sig') as f:
            reader = csv.DictReader(f)
            for row in reader:
                shop_url = row.get("shop_url", "").strip()
                if shop_url:
                    existing_shops.add(shop_url)
        print(f"既存のショップURL数: {len(existing_shops)}")
    else:
        print("既存の企業リストが見つかりません。")

    # 1) 対象の楽天カテゴリ／検索結果ページのURLを入力
    print("例: https://www.rakuten.co.jp/category/110729/ または https://search.rakuten.co.jp/search/mall/レディースファッション")
    while True:
        input_url = input("スクレイピング対象URLを入力してください: ").strip()
        if not input_url.startswith("https://"):
            print("URLが不正です。先頭がhttps:// で始まる必要があります。\n")
            continue
        else:
            print("スクレイピングを開始します。巡回範囲が大きくなる可能性があるのでご注意ください。")
            break

    scraper = RakutenScraper(input_url)

    # 2) 初回リクエストでリンクを取得
    first_links = scraper.simple_request(input_url, first=True)
    if not first_links:
        print("初回のリクエストでリンクが取得できませんでした。終了します。")
        sys.exit(0)

    # 3) 初回で取得したリンクをさらに巡回してリンクを収集（BFS的に1階層のみ）
    #    ※ユーザーが途中で中断しても、これまでに集まったリンクで後続処理を行います。
    try:
        start_time = time.time()
        for link in first_links:
            # ここでは30分を上限として巡回
            if time.time() - start_time > 1800:
                print("30分経過したため、巡回を終了します。")
                break
            if link.startswith("http"):
                scraper.simple_request(link, first=False)
                time.sleep(random.uniform(1, 1.7))
                # 毎回、商品ページURLとショップURLを更新してチェック
                scraper.get_item_url()
                current_shop_urls = scraper.extract_shop_urls(existing_shops)
                print(f"現在の新規ショップURL数: {len(current_shop_urls)}")
    except KeyboardInterrupt:
        print("\nユーザーによって中断されました。これまでのデータを元に後続の処理を実行します。")
    finally:
        # 最終的に再計算して全てのショップURLを確認（重複は除外済み）
        item_urls = scraper.get_item_url()
        shop_urls = scraper.extract_shop_urls(existing_shops)
        print(f"\n最終的な商品ページURL数: {len(item_urls)}")
        print(f"取得した新規ショップURLの数: {len(shop_urls)}")

        # 4) 各ショップURLに対して末尾にinfo.htmlを付与し、
        #    企業情報ページから企業名と電話番号を取得
        results = []
        for shop_url in shop_urls:
            info_url = shop_url.rstrip('/') + '/info.html'
            company_name, telephone = get_company_info_from_info_page(shop_url)
            results.append({
                "shop_url": shop_url,         # info.html付与前のショップURL
                "info_url": info_url,           # info.htmlを付与したURL
                "company_name": company_name,
                "telephone": telephone
            })
            time.sleep(random.uniform(1, 2))
        
        # 5) CSV保存用ディレクトリの作成
        csv_dir = "./csv_data/"
        if not os.path.exists(csv_dir):
            os.makedirs(csv_dir)
        
        # 6) 新たに抽出した企業情報をCSVに書き出し（ショップURL、info_url、企業名、TEL）
        filename = f"{today}-company_info.csv"
        output_path = os.path.join(csv_dir, filename)
        
        # Excelで開いたときに文字化けを防ぐため、BOM付きUTF-8で保存
        with open(output_path, 'w', newline='', encoding='utf-8-sig') as f:
            fieldnames = ["shop_url", "info_url", "company_name", "telephone"]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for row in results:
                writer.writerow(row)
        
        print(f"\n--- CSV出力完了 ---\nファイル: {output_path}")
        
        # 既存の企業リスト（ショップURLのみ）に新たなショップURLを追加して更新
        all_shops = existing_shops.union({res["shop_url"] for res in results})
        with open(existing_companies_file, 'w', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=["shop_url"])
            writer.writeheader()
            for shop in all_shops:
                writer.writerow({"shop_url": shop})
        
        print(f"\n--- 既存企業リストの更新完了 ---\nファイル: {existing_companies_file}")
        print("処理が完了しました。")

既存の企業リスト ./rakuten_scraping.csv を読み込みます。
既存のショップURL数: 0
例: https://www.rakuten.co.jp/category/110729/ または https://search.rakuten.co.jp/search/mall/レディースファッション


スクレイピング対象URLを入力してください:  https://search.rakuten.co.jp/search/mall/-/101975/?min=10000


スクレイピングを開始します。巡回範囲が大きくなる可能性があるのでご注意ください。
アクセス中: https://search.rakuten.co.jp/search/mall/-/101975/?min=10000 | 既存リンク数: 0
アクセス中: https://www.rakuten.co.jp/ | 既存リンク数: 284
現在の新規ショップURL数: 5
アクセス中: https://basket.step.rakuten.co.jp/rms/mall/bs/cartall/ | 既存リンク数: 832
現在の新規ショップURL数: 5
アクセス中: https://event.rakuten.co.jp/notification/ | 既存リンク数: 837
現在の新規ショップURL数: 5
アクセス中: https://coupon.rakuten.co.jp/myCoupon/%E6%A5%BD%E5%A4%A9%E5%B8%82%E5%A0%B4 | 既存リンク数: 850
現在の新規ショップURL数: 5
アクセス中: https://ashiato.rakuten.co.jp/rms/sd/ashiato/vc | 既存リンク数: 850
現在の新規ショップURL数: 5
アクセス中: https://my.bookmark.rakuten.co.jp/item | 既存リンク数: 856
現在の新規ショップURL数: 5
アクセス中: https://order.my.rakuten.co.jp/ | 既存リンク数: 856
現在の新規ショップURL数: 5
アクセス中: https://ichiba.faq.rakuten.net/ | 既存リンク数: 856
現在の新規ショップURL数: 5
アクセス中: https://ichiba.faq.rakuten.net/form/item-guide | 既存リンク数: 912
現在の新規ショップURL数: 5
アクセス中: https://www.rakuten.co.jp/ec/?camp=701100000008vIp&scid=wi_ich_common | 既存リンク数: 914


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


現在の新規ショップURL数: 5
アクセス中: https://event.rakuten.co.jp/campaign/supersale/?l-id=search_browse_pc_header_top | 既存リンク数: 990
現在の新規ショップURL数: 5
アクセス中: https://www.rakuten.co.jp | 既存リンク数: 1137
現在の新規ショップURL数: 5
アクセス中: https://search.rakuten.co.jp/search/mall/-/101070/?min=10000 | 既存リンク数: 1137
現在の新規ショップURL数: 10
アクセス中: https://search.rakuten.co.jp/search/mall/-/201887/?min=10000 | 既存リンク数: 1369
現在の新規ショップURL数: 11
アクセス中: https://search.rakuten.co.jp/search/mall/-/201874/?min=10000 | 既存リンク数: 1575
現在の新規ショップURL数: 13
アクセス中: https://search.rakuten.co.jp/search/mall/-/402289/?min=10000 | 既存リンク数: 1784
現在の新規ショップURL数: 16
アクセス中: https://search.rakuten.co.jp/search/mall/-/201883/?min=10000 | 既存リンク数: 2011
現在の新規ショップURL数: 19
アクセス中: https://search.rakuten.co.jp/search/mall/-/201877/?min=10000 | 既存リンク数: 2280
現在の新規ショップURL数: 19
アクセス中: https://search.rakuten.co.jp/search/mall/-/565745/?min=10000 | 既存リンク数: 2473
現在の新規ショップURL数: 24
アクセス中: https://search.rakuten.co.jp/search/mall/-/565746/?min=10000 | 既存リンク数: 2734
現在の新規ショップ

C:\Users\to0ar\AppData\Local\Temp\ipykernel_6584\1487977519.py:123: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tel_elem = soup.find(text=re.compile("TEL:"))


取得する企業情報ページ: https://www.rakuten.co.jp/f182010-fukui/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/kaguin/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/webbymono/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/euro-surplus/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/esports/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/field-seven/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/victorinox/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/twopedal/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/enetroom/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/nuuca/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/f212105-ena/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/wild-1/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/manpyousyouten0802/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/e-kurashi/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/sportsx/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/oohashitent/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/lbreath/in

PermissionError: [Errno 13] Permission denied: './csv_data/2025-03-10-company_info.csv'

In [13]:
import os
print(os.getcwd())

C:\Users\to0ar\PROLE


In [10]:
import requests
import time
import datetime
import random
from bs4 import BeautifulSoup
import os
import sys
import re
import csv

today = str(datetime.date.today())

class RakutenScraper:
    """
    楽天のカテゴリページや検索結果ページから再帰的にリンクを収集し、
    商品ページURLから「ショップURL」を抽出するサンプルクラス。
    できるだけ多くのショップURLを抽出します。
    """
    def __init__(self, base_url):
        self.base_url = base_url
        self.master_list = []        # 再帰的に収集したリンクを保存
        self.item_page_list = []     # 楽天の商品ページURLのみを格納
        self.shop_url_list = set()   # ショップURL（重複排除）を格納

    def simple_request(self, url, first=False):
        """
        指定URLのページから <a> タグの href を収集し、self.master_list に追加する。
        first=True のときは、取得したリンクリストを返す（初回のみ）。
        """
        print(f"アクセス中: {url} | 既存リンク数: {len(self.master_list)}")
        try:
            r = requests.get(url, timeout=(30.0, 47.5))
        except requests.exceptions.RequestException:
            return False

        if r.status_code == 200 and 'text/html' in r.headers.get('Content-Type', ''):
            soup = BeautifulSoup(r.content, 'html.parser')
            atag_list = soup.find_all('a')
            if first:
                unique_hrefs = []
                for a_tag in atag_list:
                    href = a_tag.get('href')
                    if href and (href not in self.master_list):
                        self.master_list.append(href)
                    if href and (href not in unique_hrefs):
                        unique_hrefs.append(href)
                return unique_hrefs
            else:
                for a_tag in atag_list:
                    href = a_tag.get('href')
                    if href and (href not in self.master_list):
                        self.master_list.append(href)
                return len(self.master_list)
        else:
            return False

    def get_item_url(self):
        """
        master_list に含まれるURLのうち、楽天の商品ページURLを抽出し、
        self.item_page_list を再計算する。
        例: https://item.rakuten.co.jp/ショップコード/商品ID/
        """
        pattern = r'^https://item\.rakuten\.co\.jp/([^/]+)/(\d+)/?$'
        self.item_page_list = []
        for url in self.master_list:
            if url and re.match(pattern, url):
                self.item_page_list.append(url)
        return self.item_page_list

    def extract_shop_urls(self, existing_shops=None):
        """
        item_page_list の各URLからショップコードを抜き出し、
        https://www.rakuten.co.jp/ショップコード/ の形でショップURLを生成します。
        既存のショップURL（existing_shops）と重複していない場合のみ追加します。
        """
        pattern = r'^https://item\.rakuten\.co\.jp/([^/]+)/(\d+)/?$'
        shop_urls = set()
        for item_url in self.item_page_list:
            m = re.match(pattern, item_url)
            if m:
                shop_code = m.group(1)
                shop_url = f"https://www.rakuten.co.jp/{shop_code}/"
                if existing_shops is None or shop_url not in existing_shops:
                    shop_urls.add(shop_url)
        self.shop_url_list = shop_urls
        return list(shop_urls)

def get_company_info_from_info_page(shop_url):
    """
    ショップURLに対して末尾にinfo.htmlを付与し、
    固定の企業情報ページHTMLから、<dl>内の<dt>タグのテキストを取得します。
    そのテキスト内で「株式会社」を含む部分を抽出し、
    ※「有限会社」が含まれているものは抽出対象外とします。
    TEL:の後の電話番号も抽出します。
    """
    info_url = shop_url.rstrip('/') + '/info.html'
    print(f"取得する企業情報ページ: {info_url}")
    
    try:
        response = requests.get(info_url, timeout=15)
        response.raise_for_status()
    except Exception as e:
        print("企業情報ページの取得に失敗しました:", e)
        return "Not Found", "Not Found"
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    company_name = "Not Found"
    phone_number = "Not Found"
    
    dl_tag = soup.find("dl")
    if dl_tag:
        dt_tags = dl_tag.find_all("dt")
        for dt in dt_tags:
            dt_text = dt.get_text(" ", strip=True)
            if not dt_text:
                continue
            # 「株式会社」を含む部分を、余計な情報（〒やTEL等）が出現する前まで抽出
            pattern = r'(.*?株式会社.*?)(?=〒|TEL:|FAX:|代表者:|店舗運営責任者:|店舗セキュリティ責任者:|購入履歴|$)'
            match = re.search(pattern, dt_text)
            if match:
                company_name = match.group(1).strip()
                break

    tel_elem = soup.find(text=re.compile("TEL:"))
    if tel_elem:
        match = re.search(r'TEL:\s*([\d\-]+)', tel_elem)
        if match:
            phone_number = match.group(1)
    
    return company_name, phone_number

if __name__ == "__main__":
    # 既存の企業リスト（ショップURLのみを記載したCSV）のファイルパス（相対パス）
    existing_companies_file = "./extracted_companies.csv"
    existing_shops = set()
    if os.path.exists(existing_companies_file):
        print(f"既存の企業リスト {existing_companies_file} を読み込みます。")
        with open(existing_companies_file, 'r', encoding='utf-8-sig') as f:
            reader = csv.DictReader(f)
            for row in reader:
                shop_url = row.get("shop_url", "").strip()
                if shop_url:
                    existing_shops.add(shop_url)
        print(f"既存のショップURL数: {len(existing_shops)}")
    else:
        print("既存の企業リストが見つかりません。")

    print("例: https://www.rakuten.co.jp/category/110729/ または https://search.rakuten.co.jp/search/mall/レディースファッション")
    while True:
        input_url = input("スクレイピング対象URLを入力してください: ").strip()
        if not input_url.startswith("https://"):
            print("URLが不正です。先頭がhttps:// で始まる必要があります。\n")
            continue
        else:
            print("スクレイピングを開始します。巡回範囲が大きくなる可能性があるのでご注意ください。")
            break

    scraper = RakutenScraper(input_url)

    first_links = scraper.simple_request(input_url, first=True)
    if not first_links:
        print("初回のリクエストでリンクが取得できませんでした。終了します。")
        sys.exit(0)

    try:
        start_time = time.time()
        for link in first_links:
            if time.time() - start_time > 1800:
                print("30分経過したため、巡回を終了します。")
                break
            if link.startswith("http"):
                scraper.simple_request(link, first=False)
                time.sleep(random.uniform(1, 1.7))
                scraper.get_item_url()
                current_shop_urls = scraper.extract_shop_urls(existing_shops)
                print(f"現在の新規ショップURL数: {len(current_shop_urls)}")
    except KeyboardInterrupt:
        print("\nユーザーによって中断されました。これまでのデータを元に後続の処理を実行します。")
    finally:
        item_urls = scraper.get_item_url()
        shop_urls = scraper.extract_shop_urls(existing_shops)
        print(f"\n最終的な商品ページURL数: {len(item_urls)}")
        print(f"取得した新規ショップURLの数: {len(shop_urls)}")

        results = []
        for shop_url in shop_urls:
            info_url = shop_url.rstrip('/') + '/info.html'
            company_name, telephone = get_company_info_from_info_page(shop_url)
            results.append({
                "shop_url": shop_url,
                "info_url": info_url,
                "company_name": company_name,
                "telephone": telephone
            })
            time.sleep(random.uniform(1, 2))
        
        csv_dir = "./csv_data/"
        if not os.path.exists(csv_dir):
            os.makedirs(csv_dir)
        
        filename = f"{today}-company_info.csv"
        output_path = os.path.join(csv_dir, filename)
        
        with open(output_path, 'w', newline='', encoding='utf-8-sig') as f:
            fieldnames = ["shop_url", "info_url", "company_name", "telephone"]
            writer = csv.DictWriter(f, fieldnames=fieldnames)
            writer.writeheader()
            for row in results:
                writer.writerow(row)
        
        print(f"\n--- CSV出力完了 ---\nファイル: {output_path}")
        
        # 既存の企業リストに新規抽出されたショップURLのみを追記する（既存の内容は保持）
        new_shops = {res["shop_url"] for res in results} - existing_shops
        if new_shops:
            with open(existing_companies_file, 'a', newline='', encoding='utf-8-sig') as f:
                writer = csv.DictWriter(f, fieldnames=["shop_url"])
                for shop in new_shops:
                    writer.writerow({"shop_url": shop})
            print(f"\n--- 新規ショップURLを既存企業リストに追加しました ---\nファイル: {existing_companies_file}")
        else:
            print("新しいショップURLは見つかりませんでした。")
        
        print("処理が完了しました。")

既存の企業リストが見つかりません。
例: https://www.rakuten.co.jp/category/110729/ または https://search.rakuten.co.jp/search/mall/レディースファッション


スクレイピング対象URLを入力してください:  https://search.rakuten.co.jp/search/mall/-/565210/?min=10000


スクレイピングを開始します。巡回範囲が大きくなる可能性があるのでご注意ください。
アクセス中: https://search.rakuten.co.jp/search/mall/-/565210/?min=10000 | 既存リンク数: 0
アクセス中: https://www.rakuten.co.jp/ | 既存リンク数: 475
現在の新規ショップURL数: 4
アクセス中: https://basket.step.rakuten.co.jp/rms/mall/bs/cartall/ | 既存リンク数: 1023
現在の新規ショップURL数: 4
アクセス中: https://event.rakuten.co.jp/notification/ | 既存リンク数: 1028
現在の新規ショップURL数: 4
アクセス中: https://coupon.rakuten.co.jp/myCoupon/%E6%A5%BD%E5%A4%A9%E5%B8%82%E5%A0%B4 | 既存リンク数: 1041
現在の新規ショップURL数: 4
アクセス中: https://ashiato.rakuten.co.jp/rms/sd/ashiato/vc | 既存リンク数: 1041
現在の新規ショップURL数: 4
アクセス中: https://my.bookmark.rakuten.co.jp/item | 既存リンク数: 1047
現在の新規ショップURL数: 4
アクセス中: https://order.my.rakuten.co.jp/ | 既存リンク数: 1047
現在の新規ショップURL数: 4
アクセス中: https://ichiba.faq.rakuten.net/ | 既存リンク数: 1047
現在の新規ショップURL数: 4
アクセス中: https://ichiba.faq.rakuten.net/form/item-guide | 既存リンク数: 1103
現在の新規ショップURL数: 4
アクセス中: https://www.rakuten.co.jp/ec/?camp=701100000008vIp&scid=wi_ich_common | 既存リンク数: 1105


Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


現在の新規ショップURL数: 4
アクセス中: https://event.rakuten.co.jp/campaign/supersale/?l-id=search_browse_pc_header_top | 既存リンク数: 1181
現在の新規ショップURL数: 4
アクセス中: https://www.rakuten.co.jp | 既存リンク数: 1328
現在の新規ショップURL数: 4
アクセス中: https://search.rakuten.co.jp/search/mall/-/216131/?min=10000 | 既存リンク数: 1328
現在の新規ショップURL数: 5
アクセス中: https://search.rakuten.co.jp/search/mall/-/565210/tg5002044/?min=10000 | 既存リンク数: 1667
現在の新規ショップURL数: 7
アクセス中: https://search.rakuten.co.jp/search/mall/-/565210/tg5002043/?min=10000 | 既存リンク数: 2009
現在の新規ショップURL数: 11
アクセス中: https://search.rakuten.co.jp/search/mall/-/110933/?min=10000 | 既存リンク数: 2375
現在の新規ショップURL数: 11
アクセス中: https://search.rakuten.co.jp/search/mall/-/301577/?min=10000 | 既存リンク数: 2685
現在の新規ショップURL数: 11
アクセス中: https://search.rakuten.co.jp/search/mall/-/100472/?min=10000 | 既存リンク数: 3001
現在の新規ショップURL数: 11
アクセス中: https://search.rakuten.co.jp/search/mall/-/110974/?min=10000 | 既存リンク数: 3351
現在の新規ショップURL数: 11
アクセス中: https://search.rakuten.co.jp/search/mall/-/565213/?min=10000 | 既存リ

C:\Users\to0ar\AppData\Local\Temp\ipykernel_6584\1735336263.py:125: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tel_elem = soup.find(text=re.compile("TEL:"))


取得する企業情報ページ: https://www.rakuten.co.jp/ray-g-cast/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/sim03/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/perenne/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/tabitora/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/hbespoke/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/aikobo/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/esports/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/cocotrip/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/f332020-kurashiki/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/hypermarket/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/arista/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/lag-onlinestore/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/kinoco/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/yeppuda/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/ace-store/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/auc-marks-run/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/lesnoirgraph/info.

In [15]:
import requests
import time
import datetime
import random
from bs4 import BeautifulSoup
import os
import sys
import re
import csv
from urllib.parse import urljoin

today = str(datetime.date.today())

def get_shop_id(shop_url):
    """
    ショップURL（例: https://www.rakuten.co.jp/SHOPID/）からSHOPIDを抽出します。
    """
    pattern = r'^https://www\.rakuten\.co\.jp/([^/]+)/?$'
    m = re.match(pattern, shop_url)
    if m:
        return m.group(1)
    else:
        return None

class RakutenScraper:
    """
    楽天のカテゴリページや検索結果ページから再帰的にリンクを収集し、
    商品ページURLからショップURLを抽出するサンプルクラスです。
    """
    def __init__(self, base_url):
        self.base_url = base_url
        self.master_list = []        # 収集した全リンク
        self.item_page_list = []     # 楽天の商品ページURLのみ
        self.shop_url_list = set()   # 重複排除済みショップURL

    def simple_request(self, url, first=False):
        """
        指定されたURLのページから <a> タグの href を収集し、self.master_list に追加します。
        first=True の場合、初回に取得したリンクリストを返します。
        """
        print(f"アクセス中: {url} | 既存リンク数: {len(self.master_list)}")
        try:
            r = requests.get(url, timeout=(30.0, 47.5))
        except requests.exceptions.RequestException:
            return False

        if r.status_code == 200 and 'text/html' in r.headers.get('Content-Type', ''):
            soup = BeautifulSoup(r.content, 'html.parser')
            atag_list = soup.find_all('a')
            if first:
                unique_hrefs = []
                for a_tag in atag_list:
                    href = a_tag.get('href')
                    if href and (href not in self.master_list):
                        self.master_list.append(href)
                    if href and (href not in unique_hrefs):
                        unique_hrefs.append(href)
                return unique_hrefs
            else:
                for a_tag in atag_list:
                    href = a_tag.get('href')
                    if href and (href not in self.master_list):
                        self.master_list.append(href)
                return len(self.master_list)
        else:
            return False

    def get_item_url(self):
        """
        master_list の中から楽天の商品ページURL（例: https://item.rakuten.co.jp/SHOPID/商品ID/）を抽出し、
        self.item_page_list に再格納します。
        """
        pattern = r'^https://item\.rakuten\.co\.jp/([^/]+)/(\d+)/?$'
        self.item_page_list = []
        for url in self.master_list:
            if url and re.match(pattern, url):
                self.item_page_list.append(url)
        return self.item_page_list

    def extract_shop_urls(self, existing_shop_ids=None):
        """
        item_page_list の各URLからショップIDを抽出し、
        https://www.rakuten.co.jp/SHOPID/ の形式のショップURLを生成します。
        既存のショップID（existing_shop_ids）に含まれていなければ追加します。
        """
        pattern = r'^https://item\.rakuten\.co\.jp/([^/]+)/(\d+)/?$'
        shop_urls = set()
        for item_url in self.item_page_list:
            m = re.match(pattern, item_url)
            if m:
                shop_id = m.group(1)
                shop_url = f"https://www.rakuten.co.jp/{shop_id}/"
                if existing_shop_ids is None or shop_id not in existing_shop_ids:
                    shop_urls.add(shop_url)
        self.shop_url_list = shop_urls
        return list(shop_urls)

def get_company_info_from_info_page(shop_url):
    """
    指定のショップURLに末尾の info.html を付与して企業情報ページにアクセスし、
    <dl> 内の <dt> タグからテキストを取得します。
    そのテキスト内で「株式会社」を含む部分を、余計な情報（〒, TEL, FAX, 代表者, etc.）が出現する直前まで抽出し企業名とします。
    ※「有限会社」が含まれるものは対象外です。
    また、TEL: に続く電話番号も抽出します。
    """
    info_url = shop_url.rstrip('/') + '/info.html'
    print(f"取得する企業情報ページ: {info_url}")
    
    try:
        response = requests.get(info_url, timeout=15)
        response.raise_for_status()
    except Exception as e:
        print("企業情報ページの取得に失敗しました:", e)
        return "Not Found", "Not Found"
    
    soup = BeautifulSoup(response.content, 'html.parser')
    company_name = "Not Found"
    phone_number = "Not Found"
    
    dl_tag = soup.find("dl")
    if dl_tag:
        dt_tags = dl_tag.find_all("dt")
        for dt in dt_tags:
            dt_text = dt.get_text(" ", strip=True)
            if not dt_text:
                continue
            # 正規表現で「株式会社」を含む部分を、〒やTEL等の出現直前まで抽出
            pattern = r'(.*?株式会社.*?)(?=〒|TEL:|FAX:|代表者:|店舗運営責任者:|店舗セキュリティ責任者:|購入履歴|$)'
            match = re.search(pattern, dt_text)
            if match:
                company_name = match.group(1).strip()
                break

    tel_elem = soup.find(text=re.compile("TEL:"))
    if tel_elem:
        match = re.search(r'TEL:\s*([\d\-]+)', tel_elem)
        if match:
            phone_number = match.group(1)
    
    return company_name, phone_number

def crawl_pagination(scraper, start_url, max_pages=10):
    """
    start_url から始め、最大 max_pages ページまで巡回しながらリンクを収集します。
    HTML内の次ページリンクは、class="item-next nextPage" を持つ <a> タグで取得します。
    """
    current_url = start_url
    pages_crawled = 0
    while current_url and pages_crawled < max_pages:
        print(f"【ページ {pages_crawled+1} をクロール中】 {current_url}")
        # 現在のページのリンクを収集
        scraper.simple_request(current_url, first=True)
        time.sleep(random.uniform(1, 1.5))
        
        try:
            r = requests.get(current_url, timeout=15)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, 'html.parser')
        except Exception as e:
            print("ページ遷移用リンクの取得に失敗:", e)
            break

        # 次ページリンクを class 属性で検索
        next_link = soup.find('a', class_="item-next nextPage")
        if next_link and next_link.get('href'):
            next_url = urljoin(current_url, next_link.get('href'))
            current_url = next_url
            pages_crawled += 1
        else:
            print("次のページが見つかりません。")
            break
    print(f"合計 {pages_crawled} ページをクロールしました。")

if __name__ == "__main__":
    # 既存の企業リスト（ショップURLのみ記載）CSVのファイルパス（相対パス）
    existing_companies_file = "./rakuten_scraping.csv"
    existing_shop_ids = set()
    if os.path.exists(existing_companies_file):
        print(f"既存の企業リスト {existing_companies_file} を読み込みます。")
        with open(existing_companies_file, 'r', encoding='utf-8-sig') as f:
            reader = csv.DictReader(f)
            for row in reader:
                shop_url = row.get("shop_url", "").strip()
                shop_id = get_shop_id(shop_url)
                if shop_id:
                    existing_shop_ids.add(shop_id)
        print(f"既存のショップID数: {len(existing_shop_ids)}")
    else:
        print("既存の企業リストが見つかりません。")

    # クロール開始前の入力
    print("例: https://www.rakuten.co.jp/category/110729/ または https://search.rakuten.co.jp/search/mall/レディースファッション")
    while True:
        input_url = input("スクレイピング対象URLを入力してください: ").strip()
        if not input_url.startswith("https://"):
            print("URLが不正です。先頭がhttps:// で始まる必要があります。\n")
            continue
        else:
            print("スクレイピングを開始します。巡回範囲が大きくなる可能性があるのでご注意ください。")
            break

    scraper = RakutenScraper(input_url)
    
    # ページネーションを処理して複数ページからリンクを収集（最大 max_pages ページ）
    max_pages = 2  # 必要に応じて変更
    crawl_pagination(scraper, input_url, max_pages)

    # 収集した全リンクから商品ページURLを抽出
    item_urls = scraper.get_item_url()
    print(f"\n最終的な商品ページURL数: {len(item_urls)}")
    
    # 商品ページURLからショップURL（重複はショップIDでチェック）を抽出
    shop_urls = scraper.extract_shop_urls(existing_shop_ids)
    print(f"取得した新規ショップURLの数: {len(shop_urls)}")

    results = []
    for shop_url in shop_urls:
        info_url = shop_url.rstrip('/') + '/info.html'
        company_name, telephone = get_company_info_from_info_page(shop_url)
        results.append({
            "shop_url": shop_url,
            "info_url": info_url,
            "company_name": company_name,
            "telephone": telephone
        })
        time.sleep(random.uniform(1, 2))
    
    # CSV出力用ディレクトリ作成
    csv_dir = "./csv_data/"
    if not os.path.exists(csv_dir):
        os.makedirs(csv_dir)
    
    filename = f"{today}-company_info.csv"
    output_path = os.path.join(csv_dir, filename)
    
    with open(output_path, 'w', newline='', encoding='utf-8-sig') as f:
        fieldnames = ["shop_url", "info_url", "company_name", "telephone"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in results:
            writer.writerow(row)
    
    print(f"\n--- CSV出力完了 ---\nファイル: {output_path}")
    
    # 既存リストへの新規ショップURLの追記（ショップIDで重複チェック）
    new_shops = {res["shop_url"] for res in results if get_shop_id(res["shop_url"]) not in existing_shop_ids}
    if new_shops:
        with open(existing_companies_file, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=["shop_url"])
            for shop in new_shops:
                writer.writerow({"shop_url": shop})
        print(f"\n--- 新規ショップURLを既存企業リストに追加しました ---\nファイル: {existing_companies_file}")
    else:
        print("新しいショップURLは見つかりませんでした。")
    
    print("処理が完了しました。")

既存の企業リスト ./rakuten_scraping.csv を読み込みます。
既存のショップID数: 0
例: https://www.rakuten.co.jp/category/110729/ または https://search.rakuten.co.jp/search/mall/レディースファッション


スクレイピング対象URLを入力してください:  https://search.rakuten.co.jp/search/mall/-/101975/?min=10000


スクレイピングを開始します。巡回範囲が大きくなる可能性があるのでご注意ください。
【ページ 1 をクロール中】 https://search.rakuten.co.jp/search/mall/-/101975/?min=10000
アクセス中: https://search.rakuten.co.jp/search/mall/-/101975/?min=10000 | 既存リンク数: 0
次のページが見つかりません。
合計 0 ページをクロールしました。

最終的な商品ページURL数: 7
取得した新規ショップURLの数: 5
取得する企業情報ページ: https://www.rakuten.co.jp/e-kurashi/info.html


C:\Users\to0ar\AppData\Local\Temp\ipykernel_6584\2179025888.py:134: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tel_elem = soup.find(text=re.compile("TEL:"))


取得する企業情報ページ: https://www.rakuten.co.jp/tansu/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/unidy/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/nuuca/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/esports/info.html

--- CSV出力完了 ---
ファイル: ./csv_data/2025-03-10-company_info.csv

--- 新規ショップURLを既存企業リストに追加しました ---
ファイル: ./rakuten_scraping.csv
処理が完了しました。


In [20]:
import requests
import time
import datetime
import random
from bs4 import BeautifulSoup
import os
import sys
import re
import csv
from urllib.parse import urljoin

today = str(datetime.date.today())

def get_shop_id(shop_url):
    """
    ショップURL（例: https://www.rakuten.co.jp/SHOPID/）からSHOPIDを抽出します。
    """
    pattern = r'^https://www\.rakuten\.co\.jp/([^/]+)/?$'
    m = re.match(pattern, shop_url)
    if m:
        return m.group(1)
    else:
        return None

class RakutenScraper:
    """
    楽天のカテゴリページや検索結果ページから再帰的にリンクを収集し、
    商品ページURLからショップURLを抽出するサンプルクラスです。
    """
    def __init__(self, base_url):
        self.base_url = base_url
        self.master_list = []        # 収集した全リンク
        self.item_page_list = []     # 楽天の商品ページURLのみ
        self.shop_url_list = set()   # 重複排除済みショップURL

    def simple_request(self, url, first=False):
        """
        指定されたURLのページから <a> タグの href を収集し、self.master_list に追加します。
        first=True の場合、初回に取得したリンクリストを返します。
        """
        print(f"アクセス中: {url} | 既存リンク数: {len(self.master_list)}")
        try:
            r = requests.get(url, timeout=(30.0, 47.5))
        except requests.exceptions.RequestException:
            return False

        if r.status_code == 200 and 'text/html' in r.headers.get('Content-Type', ''):
            soup = BeautifulSoup(r.content, 'html.parser')
            atag_list = soup.find_all('a')
            if first:
                unique_hrefs = []
                for a_tag in atag_list:
                    href = a_tag.get('href')
                    if href and (href not in self.master_list):
                        self.master_list.append(href)
                    if href and (href not in unique_hrefs):
                        unique_hrefs.append(href)
                return unique_hrefs
            else:
                for a_tag in atag_list:
                    href = a_tag.get('href')
                    if href and (href not in self.master_list):
                        self.master_list.append(href)
                return len(self.master_list)
        else:
            return False

    def get_item_url(self):
        """
        master_list の中から楽天の商品ページURL（例: https://item.rakuten.co.jp/SHOPID/商品ID/）を抽出し、
        self.item_page_list に再格納します。
        """
        pattern = r'^https://item\.rakuten\.co\.jp/([^/]+)/(\d+)/?$'
        self.item_page_list = []
        for url in self.master_list:
            if url and re.match(pattern, url):
                self.item_page_list.append(url)
        return self.item_page_list

    def extract_shop_urls(self, existing_shop_ids=None):
        """
        item_page_list の各URLからショップIDを抽出し、
        https://www.rakuten.co.jp/SHOPID/ の形式のショップURLを生成します。
        既存のショップID（existing_shop_ids）に含まれていなければ追加します。
        """
        pattern = r'^https://item\.rakuten\.co\.jp/([^/]+)/(\d+)/?$'
        shop_urls = set()
        for item_url in self.item_page_list:
            m = re.match(pattern, item_url)
            if m:
                shop_id = m.group(1)
                shop_url = f"https://www.rakuten.co.jp/{shop_id}/"
                if existing_shop_ids is None or shop_id not in existing_shop_ids:
                    shop_urls.add(shop_url)
        self.shop_url_list = shop_urls
        return list(shop_urls)

def get_company_info_from_info_page(shop_url):
    """
    指定のショップURLに末尾の info.html を付与して企業情報ページにアクセスし、
    <dl> 内の <dt> タグからテキストを取得します。
    そのテキスト内で「株式会社」を含む部分を、余計な情報（〒, TEL, FAX, 代表者, etc.）が出現する直前まで抽出し企業名とします。
    ※「有限会社」が含まれるものは対象外です。
    また、TEL: に続く電話番号も抽出します。
    """
    info_url = shop_url.rstrip('/') + '/info.html'
    print(f"取得する企業情報ページ: {info_url}")
    
    try:
        response = requests.get(info_url, timeout=15)
        response.raise_for_status()
    except Exception as e:
        print("企業情報ページの取得に失敗しました:", e)
        return "Not Found", "Not Found"
    
    soup = BeautifulSoup(response.content, 'html.parser')
    company_name = "Not Found"
    phone_number = "Not Found"
    
    dl_tag = soup.find("dl")
    if dl_tag:
        dt_tags = dl_tag.find_all("dt")
        for dt in dt_tags:
            dt_text = dt.get_text(" ", strip=True)
            if not dt_text:
                continue
            # 正規表現で「株式会社」を含む部分を、〒やTEL等が出現する直前まで抽出
            pattern = r'(.*?株式会社.*?)(?=〒|TEL:|FAX:|代表者:|店舗運営責任者:|店舗セキュリティ責任者:|購入履歴|$)'
            match = re.search(pattern, dt_text)
            if match:
                company_name = match.group(1).strip()
                break

    tel_elem = soup.find(text=re.compile("TEL:"))
    if tel_elem:
        match = re.search(r'TEL:\s*([\d\-]+)', tel_elem)
        if match:
            phone_number = match.group(1)
    
    return company_name, phone_number

def crawl_pagination(scraper, start_url, max_pages=10):
    """
    start_url から始め、最大 max_pages ページまで巡回しながらリンクを収集します。
    HTML内の次ページリンクは、クラスが "item", "-next", "nextPage" を含む <a> タグで取得します。
    """
    current_url = start_url
    pages_crawled = 0
    while current_url and pages_crawled < max_pages:
        print(f"【ページ {pages_crawled+1} をクロール中】 {current_url}")
        # 現在のページのリンクを収集
        scraper.simple_request(current_url, first=True)
        time.sleep(random.uniform(1, 1.5))
        
        try:
            r = requests.get(current_url, timeout=15)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, 'html.parser')
        except Exception as e:
            print("ページ遷移用リンクの取得に失敗:", e)
            break

        # 次ページリンクを CSS セレクターで取得 (クラス: item, -next, nextPage)
        next_link = soup.select_one("a.item.-next.nextPage")
        if next_link and next_link.get('href'):
            next_url = urljoin(current_url, next_link.get('href'))
            current_url = next_url
            pages_crawled += 1
        else:
            print("次のページが見つかりません。")
            break
    print(f"合計 {pages_crawled} ページをクロールしました。")

if __name__ == "__main__":
    # 既存の企業リスト（ショップURLのみ記載）CSVのファイルパス（相対パス）
    existing_companies_file = "./rakuten_scraping.csv"
    existing_shop_ids = set()
    if os.path.exists(existing_companies_file):
        print(f"既存の企業リスト {existing_companies_file} を読み込みます。")
        with open(existing_companies_file, 'r', encoding='utf-8-sig') as f:
            reader = csv.DictReader(f)
            for row in reader:
                shop_url = row.get("shop_url", "").strip()
                shop_id = get_shop_id(shop_url)
                if shop_id:
                    existing_shop_ids.add(shop_id)
        print(f"既存のショップID数: {len(existing_shop_ids)}")
    else:
        print("既存の企業リストが見つかりません。")

    print("例: https://www.rakuten.co.jp/category/110729/ または https://search.rakuten.co.jp/search/mall/レディースファッション")
    while True:
        input_url = input("スクレイピング対象URLを入力してください: ").strip()
        if not input_url.startswith("https://"):
            print("URLが不正です。先頭がhttps:// で始まる必要があります。\n")
            continue
        else:
            print("スクレイピングを開始します。巡回範囲が大きくなる可能性があるのでご注意ください。")
            break

    scraper = RakutenScraper(input_url)
    
    # ページネーションを処理して複数ページからリンクを収集（最大 max_pages ページ）
    max_pages = 100  # 必要に応じて変更
    crawl_pagination(scraper, input_url, max_pages)

    # 収集した全リンクから商品ページURLを抽出
    item_urls = scraper.get_item_url()
    print(f"\n最終的な商品ページURL数: {len(item_urls)}")
    
    # 商品ページURLからショップURL（重複はショップIDでチェック）を抽出
    shop_urls = scraper.extract_shop_urls(existing_shop_ids)
    print(f"取得した新規ショップURLの数: {len(shop_urls)}")

    results = []
    for shop_url in shop_urls:
        info_url = shop_url.rstrip('/') + '/info.html'
        company_name, telephone = get_company_info_from_info_page(shop_url)
        results.append({
            "shop_url": shop_url,
            "info_url": info_url,
            "company_name": company_name,
            "telephone": telephone
        })
        time.sleep(random.uniform(1, 2))
    
    # CSV出力用ディレクトリ作成
    csv_dir = "./csv_data/"
    if not os.path.exists(csv_dir):
        os.makedirs(csv_dir)
    
    filename = f"{today}-company_info.csv"
    output_path = os.path.join(csv_dir, filename)
    
    with open(output_path, 'w', newline='', encoding='utf-8-sig') as f:
        fieldnames = ["shop_url", "info_url", "company_name", "telephone"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in results:
            writer.writerow(row)
    
    print(f"\n--- CSV出力完了 ---\nファイル: {output_path}")
    
    # 既存リストへの新規ショップURLの追記（ショップIDで重複チェック）
    new_shops = {res["shop_url"] for res in results if get_shop_id(res["shop_url"]) not in existing_shop_ids}
    if new_shops:
        with open(existing_companies_file, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=["shop_url"])
            for shop in new_shops:
                writer.writerow({"shop_url": shop})
        print(f"\n--- 新規ショップURLを既存企業リストに追加しました ---\nファイル: {existing_companies_file}")
    else:
        print("新しいショップURLは見つかりませんでした。")
    
    print("処理が完了しました。")


既存の企業リスト ./rakuten_scraping.csv を読み込みます。
既存のショップID数: 0
例: https://www.rakuten.co.jp/category/110729/ または https://search.rakuten.co.jp/search/mall/レディースファッション


スクレイピング対象URLを入力してください:  https://search.rakuten.co.jp/search/mall/-/101975/?min=10000


スクレイピングを開始します。巡回範囲が大きくなる可能性があるのでご注意ください。
【ページ 1 をクロール中】 https://search.rakuten.co.jp/search/mall/-/101975/?min=10000
アクセス中: https://search.rakuten.co.jp/search/mall/-/101975/?min=10000 | 既存リンク数: 0
【ページ 2 をクロール中】 https://search.rakuten.co.jp/search/mall/-/101975/?min=10000&p=2
アクセス中: https://search.rakuten.co.jp/search/mall/-/101975/?min=10000&p=2 | 既存リンク数: 284
【ページ 3 をクロール中】 https://search.rakuten.co.jp/search/mall/-/101975/?min=10000&p=3
アクセス中: https://search.rakuten.co.jp/search/mall/-/101975/?min=10000&p=3 | 既存リンク数: 429
【ページ 4 をクロール中】 https://search.rakuten.co.jp/search/mall/-/101975/?min=10000&p=4
アクセス中: https://search.rakuten.co.jp/search/mall/-/101975/?min=10000&p=4 | 既存リンク数: 561
【ページ 5 をクロール中】 https://search.rakuten.co.jp/search/mall/-/101975/?min=10000&p=5
アクセス中: https://search.rakuten.co.jp/search/mall/-/101975/?min=10000&p=5 | 既存リンク数: 681
【ページ 6 をクロール中】 https://search.rakuten.co.jp/search/mall/-/101975/?min=10000&p=6
アクセス中: https://search.rakuten.co.jp/search/mall/-/101975/?m

C:\Users\to0ar\AppData\Local\Temp\ipykernel_6584\474183946.py:134: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tel_elem = soup.find(text=re.compile("TEL:"))


取得する企業情報ページ: https://www.rakuten.co.jp/himarayaod/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/f182010-fukui/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/shop-senjin/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/storage-g/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/plusys7022/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/chamonix/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/euro-surplus/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/esports/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/rococostore/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/g-dreams/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/field-seven/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/ninerock/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/workwearlab/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/victorinox/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/twopedal/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/enetroom/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/leathe

In [29]:
import requests
import time
import datetime
import random
from bs4 import BeautifulSoup
import os
import sys
import re
import csv
from urllib.parse import urljoin

today = str(datetime.date.today())

def get_shop_id(shop_url):
    """
    ショップURL（例: https://www.rakuten.co.jp/SHOPID/ または https://www.rakuten.co.jp/gold/SHOPID/）からSHOPIDを抽出します。
    """
    pattern = r'^https://www\.rakuten\.co\.jp/(?:gold/)?([^/]+)/?$'
    m = re.match(pattern, shop_url)
    if m:
        return m.group(1)
    else:
        return None

class RakutenScraper:
    """
    楽天のカテゴリページや検索結果ページから再帰的にリンクを収集し、
    商品ページURLからショップURLを抽出するサンプルクラスです。
    """
    def __init__(self, base_url):
        self.base_url = base_url
        self.master_list = []        # 収集した全リンク
        self.item_page_list = []     # 楽天の商品ページURLのみ
        self.shop_url_list = set()   # 重複排除済みショップURL

    def simple_request(self, url, first=False):
        """
        指定されたURLのページから <a> タグの href を収集し、self.master_list に追加します。
        first=True の場合、初回に取得したリンクリストを返します。
        """
        print(f"アクセス中: {url} | 既存リンク数: {len(self.master_list)}")
        try:
            r = requests.get(url, timeout=(30.0, 47.5))
        except requests.exceptions.RequestException:
            return False

        if r.status_code == 200 and 'text/html' in r.headers.get('Content-Type', ''):
            soup = BeautifulSoup(r.content, 'html.parser')
            atag_list = soup.find_all('a')
            if first:
                unique_hrefs = []
                for a_tag in atag_list:
                    href = a_tag.get('href')
                    if href and (href not in self.master_list):
                        self.master_list.append(href)
                    if href and (href not in unique_hrefs):
                        unique_hrefs.append(href)
                return unique_hrefs
            else:
                for a_tag in atag_list:
                    href = a_tag.get('href')
                    if href and (href not in self.master_list):
                        self.master_list.append(href)
                return len(self.master_list)
        else:
            return False

    def get_item_url(self):
        """
        master_list の中から楽天の商品ページURL（例: https://item.rakuten.co.jp/SHOPID/商品ID/）を抽出し、
        self.item_page_list に再格納します。
        """
        pattern = r'^https://item\.rakuten\.co\.jp/([^/]+)/(\d+)/?$'
        self.item_page_list = []
        for url in self.master_list:
            if url and re.match(pattern, url):
                self.item_page_list.append(url)
        return self.item_page_list

    def extract_shop_urls(self, existing_shop_ids=None):
        """
        item_page_list の各URLからショップIDを抽出し、
        https://www.rakuten.co.jp/SHOPID/ の形式のショップURLを生成します。
        既存のショップID（existing_shop_ids）に含まれていなければ追加します。
        """
        pattern = r'^https://item\.rakuten\.co\.jp/([^/]+)/(\d+)/?$'
        shop_urls = set()
        for item_url in self.item_page_list:
            m = re.match(pattern, item_url)
            if m:
                shop_id = m.group(1)
                shop_url = f"https://www.rakuten.co.jp/{shop_id}/"
                if existing_shop_ids is None or shop_id not in existing_shop_ids:
                    shop_urls.add(shop_url)
        self.shop_url_list = shop_urls
        return list(shop_urls)

def get_company_info_from_info_page(shop_url):
    """
    指定のショップURLに末尾の info.html を付与して企業情報ページにアクセスし、
    <dl> 内の <dt> タグからテキストを取得します。
    そのテキスト内で「株式会社」を含む部分を、余計な情報（〒, TEL, FAX, 代表者, etc.）が出現する直前まで抽出し企業名とします。
    ※「有限会社」が含まれるものは対象外です。
    また、TEL: に続く電話番号も抽出します。
    """
    info_url = shop_url.rstrip('/') + '/info.html'
    print(f"取得する企業情報ページ: {info_url}")
    
    try:
        response = requests.get(info_url, timeout=15)
        response.raise_for_status()
    except Exception as e:
        print("企業情報ページの取得に失敗しました:", e)
        return "Not Found", "Not Found"
    
    soup = BeautifulSoup(response.content, 'html.parser')
    company_name = "Not Found"
    phone_number = "Not Found"
    
    dl_tag = soup.find("dl")
    if dl_tag:
        dt_tags = dl_tag.find_all("dt")
        for dt in dt_tags:
            dt_text = dt.get_text(" ", strip=True)
            if not dt_text:
                continue
            # 正規表現で「株式会社」を含む部分を、〒やTEL等が出現する直前まで抽出
            pattern = r'(.*?株式会社.*?)(?=〒|TEL:|FAX:|代表者:|店舗運営責任者:|店舗セキュリティ責任者:|購入履歴|$)'
            match = re.search(pattern, dt_text)
            if match:
                company_name = match.group(1).strip()
                break

    tel_elem = soup.find(text=re.compile("TEL:"))
    if tel_elem:
        match = re.search(r'TEL:\s*([\d\-]+)', tel_elem)
        if match:
            phone_number = match.group(1)
    
    return company_name, phone_number

def crawl_pagination(scraper, start_url, max_pages=10):
    """
    start_url から始め、最大 max_pages ページまで巡回しながらリンクを収集します。
    HTML内の次ページリンクは、クラスが "item", "-next", "nextPage" を含む <a> タグで取得します。
    """
    current_url = start_url
    pages_crawled = 0
    while current_url and pages_crawled < max_pages:
        print(f"【ページ {pages_crawled+1} をクロール中】 {current_url}")
        # 現在のページのリンクを収集
        scraper.simple_request(current_url, first=True)
        time.sleep(random.uniform(1, 1.5))
        
        try:
            r = requests.get(current_url, timeout=15)
            r.raise_for_status()
            soup = BeautifulSoup(r.content, 'html.parser')
        except Exception as e:
            print("ページ遷移用リンクの取得に失敗:", e)
            break

        # 次ページリンクを CSS セレクターで取得 (クラス: item, -next, nextPage)
        next_link = soup.select_one("a.item.-next.nextPage")
        if next_link and next_link.get('href'):
            next_url = urljoin(current_url, next_link.get('href'))
            current_url = next_url
            pages_crawled += 1
        else:
            print("次のページが見つかりません。")
            break
    print(f"合計 {pages_crawled} ページをクロールしました。")

if __name__ == "__main__":
    # 既存の企業リスト（ショップURLのみ記載）CSVのファイルパス（相対パス）
    existing_companies_file = "./rakuten_scraping.csv"
    existing_shop_ids = set()
    if os.path.exists(existing_companies_file):
        print(f"既存の企業リスト {existing_companies_file} を読み込みます。")
        with open(existing_companies_file, 'r', encoding='utf-8-sig') as f:
            reader = csv.DictReader(f)
            for row in reader:
                shop_url = row.get("shop_url", "").strip()
                shop_id = get_shop_id(shop_url)
                if shop_id:
                    existing_shop_ids.add(shop_id)
        print(f"既存のショップID数: {len(existing_shop_ids)}")
    else:
        print("既存の企業リストが見つかりません。")

    print("例: https://www.rakuten.co.jp/category/110729/ または https://search.rakuten.co.jp/search/mall/レディースファッション")
    while True:
        input_url = input("スクレイピング対象URLを入力してください: ").strip()
        if not input_url.startswith("https://"):
            print("URLが不正です。先頭がhttps:// で始まる必要があります。\n")
            continue
        else:
            print("スクレイピングを開始します。巡回範囲が大きくなる可能性があるのでご注意ください。")
            break

    scraper = RakutenScraper(input_url)
    
    # ページネーションを処理して複数ページからリンクを収集（最大 max_pages ページ）
    max_pages = 45  # 必要に応じて変更
    crawl_pagination(scraper, input_url, max_pages)

    # 収集した全リンクから商品ページURLを抽出
    item_urls = scraper.get_item_url()
    print(f"\n最終的な商品ページURL数: {len(item_urls)}")
    
    # 商品ページURLからショップURL（重複はショップIDでチェック）を抽出
    shop_urls = scraper.extract_shop_urls(existing_shop_ids)
    print(f"取得した新規ショップURLの数: {len(shop_urls)}")

    results = []
    for shop_url in shop_urls:
        info_url = shop_url.rstrip('/') + '/info.html'
        company_name, telephone = get_company_info_from_info_page(shop_url)
        results.append({
            "shop_url": shop_url,
            "info_url": info_url,
            "company_name": company_name,
            "telephone": telephone
        })
        time.sleep(random.uniform(1, 2))
    
    # CSV出力用ディレクトリ作成
    csv_dir = "./csv_data/"
    if not os.path.exists(csv_dir):
        os.makedirs(csv_dir)
    
    filename = f"{today}-company_info.csv"
    output_path = os.path.join(csv_dir, filename)
    
    with open(output_path, 'w', newline='', encoding='utf-8-sig') as f:
        fieldnames = ["shop_url", "info_url", "company_name", "telephone"]
        writer = csv.DictWriter(f, fieldnames=fieldnames)
        writer.writeheader()
        for row in results:
            writer.writerow(row)
    
    print(f"\n--- CSV出力完了 ---\nファイル: {output_path}")
    
    # 既存リストへの新規ショップURLの追記（ショップIDで重複チェック）
    new_shops = {res["shop_url"] for res in results if get_shop_id(res["shop_url"]) not in existing_shop_ids}
    if new_shops:
        with open(existing_companies_file, 'a', newline='', encoding='utf-8-sig') as f:
            writer = csv.DictWriter(f, fieldnames=["shop_url"])
            for shop in new_shops:
                writer.writerow({"shop_url": shop})
        print(f"\n--- 新規ショップURLを既存企業リストに追加しました ---\nファイル: {existing_companies_file}")
    else:
        print("新しいショップURLは見つかりませんでした。")
    
    print("処理が完了しました。")

既存の企業リスト ./rakuten_scraping.csv を読み込みます。
既存のショップID数: 3619
例: https://www.rakuten.co.jp/category/110729/ または https://search.rakuten.co.jp/search/mall/レディースファッション


スクレイピング対象URLを入力してください:  https://search.rakuten.co.jp/search/mall/-/568421/?min=10000


スクレイピングを開始します。巡回範囲が大きくなる可能性があるのでご注意ください。
【ページ 1 をクロール中】 https://search.rakuten.co.jp/search/mall/-/568421/?min=10000
アクセス中: https://search.rakuten.co.jp/search/mall/-/568421/?min=10000 | 既存リンク数: 0
【ページ 2 をクロール中】 https://search.rakuten.co.jp/search/mall/-/568421/?min=10000&p=2
アクセス中: https://search.rakuten.co.jp/search/mall/-/568421/?min=10000&p=2 | 既存リンク数: 406
【ページ 3 をクロール中】 https://search.rakuten.co.jp/search/mall/-/568421/?min=10000&p=3
アクセス中: https://search.rakuten.co.jp/search/mall/-/568421/?min=10000&p=3 | 既存リンク数: 556
【ページ 4 をクロール中】 https://search.rakuten.co.jp/search/mall/-/568421/?min=10000&p=4
アクセス中: https://search.rakuten.co.jp/search/mall/-/568421/?min=10000&p=4 | 既存リンク数: 703
【ページ 5 をクロール中】 https://search.rakuten.co.jp/search/mall/-/568421/?min=10000&p=5
アクセス中: https://search.rakuten.co.jp/search/mall/-/568421/?min=10000&p=5 | 既存リンク数: 848
【ページ 6 をクロール中】 https://search.rakuten.co.jp/search/mall/-/568421/?min=10000&p=6
アクセス中: https://search.rakuten.co.jp/search/mall/-/568421/?m

C:\Users\to0ar\AppData\Local\Temp\ipykernel_6584\708723050.py:134: DeprecationWarning: The 'text' argument to find()-type methods is deprecated. Use 'string' instead.
  tel_elem = soup.find(text=re.compile("TEL:"))


取得する企業情報ページ: https://www.rakuten.co.jp/f012301-noboribetsu/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/kosaburo/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/side7/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/f182052-ono/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/auc-sanjo-tanaka/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/johnblaze/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/zeitakuya/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/coolcat/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/bmp/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/wathz/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/f012262-sunagawa/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/deal-design/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/fgkawamura/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/inden/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/galleria-annex/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/business-bugs/info.html
取得する企業情報ページ: https://www.rakuten.co.jp/

In [24]:
import csv
import os
import re

def get_shop_id(shop_url):
    """
    ショップURL（例: 
      https://www.rakuten.co.jp/SHOPID/ または 
      https://www.rakuten.co.jp/gold/SHOPID/ 
    ）からSHOPIDを抽出します。
    """
    pattern = r'^https://www\.rakuten\.co\.jp/(?:gold/)?([^/]+)/?$'
    m = re.match(pattern, shop_url)
    if m:
        return m.group(1)
    else:
        return None

existing_companies_file = "./rakuten_scraping.csv"

if os.path.exists(existing_companies_file):
    print(f"既存の企業リスト {existing_companies_file} を読み込みます。")
    existing_shop_ids = set()
    with open(existing_companies_file, 'r', encoding='utf-8-sig') as f:
        reader = csv.DictReader(f)
        for row in reader:
            shop_url = row.get("shop_url", "").strip()
            if shop_url:
                shop_id = get_shop_id(shop_url)
                print("ショップURL:", shop_url, "| ショップID:", shop_id)
                if shop_id:
                    existing_shop_ids.add(shop_id)
    print(f"既存のショップID数: {len(existing_shop_ids)}")
else:
    print("既存の企業リストが見つかりません。")

既存の企業リスト ./rakuten_scraping.csv を読み込みます。
ショップURL: https://www.rakuten.co.jp/kanko/ | ショップID: kanko
ショップURL: https://www.rakuten.co.jp/btnc/ | ショップID: btnc
ショップURL: https://www.rakuten.co.jp/seven2seven/ | ショップID: seven2seven
ショップURL: https://www.rakuten.co.jp/eclan/ | ショップID: eclan
ショップURL: https://www.rakuten.co.jp/f012181-akabira/ | ショップID: f012181-akabira
ショップURL: https://www.rakuten.co.jp/gloopy/ | ショップID: gloopy
ショップURL: https://www.rakuten.co.jp/vivostyle/ | ショップID: vivostyle
ショップURL: https://www.rakuten.co.jp/minamistore/ | ショップID: minamistore
ショップURL: https://www.rakuten.co.jp/angers/ | ショップID: angers
ショップURL: https://www.rakuten.co.jp/mzspeed-r/ | ショップID: mzspeed-r
ショップURL: https://www.rakuten.co.jp/enich-agent/ | ショップID: enich-agent
ショップURL: https://www.rakuten.co.jp/fukusumi/ | ショップID: fukusumi
ショップURL: https://www.rakuten.co.jp/bianca-rose/ | ショップID: bianca-rose
ショップURL: https://www.rakuten.co.jp/skyartjapan/ | ショップID: skyartjapan
ショップURL: https://www.rakuten.co.jp/atrium20